In [12]:
import os
import glob
import json
from datetime import datetime, timezone
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
pd.set_option('display.max_rows', None)

directory_path = './logs/dut0_ecc_bypass/logs_client'
file_pattern = 'Client_dut*'

# Use glob to find all files matching the pattern
file_paths = glob.glob(os.path.join(directory_path, file_pattern))

In [13]:
with open('crcTable.txt', 'r') as file:
    crcTable = [int(line.strip(), 16) for line in file]

Recursively checking all the log files and sorting on Unix timestamp

In [14]:
frames = []
frame_parsing_errors = 0
crc_errors = 0
filtered_frames = []
for file_path in file_paths:
    with open(file_path, 'r') as file:
        for json_line in file:
            # Parse frame
            try:
                frame = json.loads(json_line)
            except Exception as error:
                print(f"Failed decoding JSON for file {file_path}: {error}")
            frames.append(frame)

# Sort frames based on timestamp
sorted_frames = sorted(frames, key=lambda x: x["timestamp"])
parsed_frames = []

Changing Unix to Utc

In [15]:
def decode_frame(frame_bytes):
    # Desconstructing the frame
    header = frame_bytes[0]
    frame_id = frame_bytes[1]
    payload_length = frame_bytes[2]
    payload = frame_bytes[3 : 3 + payload_length]
    crc_bytes = frame_bytes[3 + payload_length : 5 + payload_length]
    tail = frame_bytes[-1]

    # Concatenating the CRC bytes into a single number
    # Assuming CRC is in big-endian format
    crc = (crc_bytes[0] << 8) | crc_bytes[1]

    # Converting payload to hex representation
    payload_hex = [hex(byte) for byte in payload]

    crc_check = check_crc(payload, payload_length, crc)

    data = None

    if crc_check is False:
        print(f"CRC Check failed!")
    else:
        data = parse_payload(payload, frame_id)

    return data

In [16]:
def check_crc(payload, payload_length, crc_value):
        INITIAL_REMAINDER = 0xFFFF
        FINAL_XOR_VALUE = 0x0000
        remainder = INITIAL_REMAINDER

        for byte in range(payload_length):
            data = payload[byte] ^ (remainder >> (16 - 8))
            remainder = crcTable[data] ^ (remainder << 8) & 0xFFFF

        return crc_value == (remainder ^ FINAL_XOR_VALUE)

In [17]:
import struct

# +--------+--------------------+-------------+---------------+
# | Format | C Type             | Python type | Standard size |
# +--------+--------------------+-------------+---------------+
# | x      | pad byte           | no value    | (7)           |
# | c      | char               | bytes       | 1             |
# | b      | signed char        | integer     | 1             |
# | B      | unsigned char      | integer     | 1             |
# | ?      | _Bool              | bool        | 1             |
# | h      | short              | integer     | 2             |
# | H      | unsigned short     | integer     | 2             |
# | i      | int                | integer     | 4             |
# | I      | unsigned int       | integer     | 4             |
# | l      | long               | integer     | 4             |
# | L      | unsigned long      | integer     | 4             |
# | q      | long long          | integer     | 8             |
# | Q      | unsigned long long | integer     | 8             |
# | n      | ssize_t            | integer     | (3)           |
# | N      | size_t             | integer     | (3)           |
# | e      | (6)                | float       | 2             |
# | f      | float              | float       | 4             |
# | d      | double             | float       | 8             |
# | s      | char[]             | bytes       | (9)           |
# | p      | char[]             | bytes       | (8)           |
# | P      | void*              | integer     | (5)           |
# +--------+--------------------+-------------+---------------+


frame_id_formatting = {
    "BBBBBBBBBBBBBBBBBBBBBBBBBBBBBBBB": 0,  # test frame
    # "II": 1,  # fini
    "IIIIIIIIIIIIIII": 1,  # fini
    "IIIIII": 16,  # exception
}


def parse_payload(data, frame_id):
    # Ensure that the data is a bytes object
    # if not isinstance(data, bytes):
    #     raise ValueError("Data must be bytes")

    # Find the format string for the given frame_id
    format_str = None
    unpacked_data = ()
    for fmt_str, id in frame_id_formatting.items():
        if id == frame_id:
            format_str = fmt_str
            break

    if format_str is None:
        raise ValueError(f"No format string found for frame ID {frame_id}")

    # Unpack the data dynamically based on the format string
    try:
        unpacked_data = struct.unpack(format_str, data)
    except struct.error as e:
        print(f"Error unpacking data with format {format_str}: {e}, data {data}")

    # # Convert bytes to string for field4 if necessary
    # field4 = field4.decode("utf-8").rstrip("\x00")

    # for field in unpacked_data:
    #     print(hex(field))

    return unpacked_data


In [18]:
for frame in sorted_frames:
    frame['timestamp'] = str(datetime.utcfromtimestamp(float(frame['timestamp'])).replace(tzinfo=timezone.utc).strftime('%Y-%m-%d %H:%M:%S.%f UTC'))
    if 'data' in frame:
        try:
            decoded_frame = decode_frame(bytes.fromhex(frame['data']))
            #CRC check failed
            if decoded_frame == None:
                parsed_frames.append({'timestamp': frame['timestamp'], 'error': "CRC check error"})
                crc_errors += 1
            # Exception
            elif len(decoded_frame) < 15:
                parsed_frames.append({'timestamp': frame['timestamp'], 'error': f"Frame parsing error. Data field too small: {decode_frame}"})
            # Data correct
            else:
                frame['data'] = decoded_frame
                parsed_frames.append(frame)
        except Exception as error:
            frame_parsing_errors += 1
            parsed_frames.append({'timestamp': frame['timestamp'], 'error': "Frame parsing error (possibly due to comm failure)"})
            #print(f"Parsing error! {error}")
    elif 'event' in frame:
        parsed_frames.append(frame)

CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check failed!
CRC Check 

Filtering out duplicate entries

In [19]:
previous_entry = None
# Filtering duplicate payloads out
for frame in parsed_frames:
    if previous_entry is None or frame['data'] != previous_entry or 'error' in frame:
        #frame['payload'] = frame.pop('data')
        filtered_frames.append(frame)



In [20]:
data_frame = []

for frame in parsed_frames:
    if 'data' in frame:
        parsed_payload = (frame['timestamp'],) + frame['data']
        keys = ['timestamp', 'total_errors', 'mcycle', 'minstret', 'imem_se', 'imem_de', 'dmem_se', 'dmem_de', 'regfile_se', 'regfile_de', 'iv', 'jump', 'branch', 'dsp_t', 'trap', 'illegal']
        bruno_keys = ['timestamp', 'cy_c', 'tm_c', 'ir_c', 'wait_ii', 'wait_if', 'wait_mc', 'load', 'store', 'wait_ls', 'branch', 'tbranch', 'imem_ecc', 'dmem_ecc', 'regfile', 'iv']
        mapping = {num: key for num, key in zip(bruno_keys, parsed_payload)}
        data_frame.append(mapping)

df = pd.DataFrame(data_frame)

In [21]:
df_no_duplicates = df.drop_duplicates(subset='cy_c', keep='first')
df_no_duplicates

""


# IV detections

In [22]:
filtered = df[df['iv'] > 0]
#filtered

KeyError: 'iv'

# IV FPR higher than 2%

scaler = MinMaxScaler()
df['imem_se'] = scaler.fit_transform(df[['imem_se']])
df['iv'] = scaler.fit_transform(df[['iv']])
df_no_duplicates = df.drop_duplicates(subset='iv', keep='first')
fpr_violation_condition = df_no_duplicates['iv'] < (0.98*df_no_duplicates['imem_se'])
fpr_violation_condition

# Double bit errors

In [ ]:
fpr_violation_condition = df['iv'] > (0.98*df['imem_ecc'])
# df[fpr_violation_condition]

Filtering based on SmartFusion2 reset. Excluding DMEM and Register file because of scrubbing and overwriting

In [ ]:
# deviation = 10000
# mask = (df['iv'] < df['iv'].shift()-deviation) | (df['iv'] < df['iv'].shift()-deviation)
# zero_rows = df.loc[(df['imem_ecc'] == 0) & (df['dmem_ecc'] == 0) & (df['regfile'] == 0) & (df['regfile'] == 0)]
# df[mask | mask.shift(-1)]

# Initialize a variable to store the rows that meet the conditions
result_rows = []
prev_row = None

# Iterate over rows
for index, row in df.iterrows():
    # Check if the subset of columns is zero in the current row
    if row['imem_ecc'] == 0 and row['dmem_ecc'] == 0 and row['regfile'] == 0 and row['iv'] == 0:
        # Check if the corresponding columns in the previous row are non-zero
        if prev_row is not None: 
            if prev_row['imem_ecc'] != 0 or prev_row['dmem_ecc'] != 0 or prev_row['regfile'] != 0 or prev_row['iv'] != 0:
                # Print the previous non-zero row and the current row
                result_rows.append(prev_row)
                result_rows.append(row)

    # Update the previous row
    prev_row = row

pf_results = pd.DataFrame(result_rows)
pf_results

,timestamp,cy_c,tm_c,ir_c,wait_ii,wait_if,wait_mc,load,store,wait_ls,branch,tbranch,imem_ecc,dmem_ecc,regfile,iv
1323,2023-12-08 18:17:07.313224 UTC,0,96470600,11242419,16267333,38158950,22063286,4381079,1426024,16196283,817756,509563,11242,0,0,9282
1324,2023-12-08 18:17:08.840984 UTC,0,96470600,11242419,16267333,38158950,22063286,4381079,1426024,16196283,817756,509563,0,0,0,0
1325,2023-12-08 18:17:09.249002 UTC,0,96470600,11242419,16267333,38158950,22063286,4381079,1426024,16196283,817756,509563,11242,0,0,9282
1326,2023-12-08 18:17:10.776862 UTC,0,96470600,11242419,16267333,38158950,22063286,4381079,1426024,16196283,817756,509563,0,0,0,0
1327,2023-12-08 18:17:11.191691 UTC,0,96470600,11242419,16267333,38158950,22063286,4381079,1426024,16196283,817756,509563,11242,0,0,9282
1328,2023-12-08 18:17:12.721105 UTC,0,96470600,11242419,16267333,38158950,22063286,4381079,1426024,16196283,817756,509563,0,0,0,0
3082,2023-12-08 19:13:53.910259 UTC,0,96470600,11242419,16267333,38158950,22063286,4381079,1426024,16196283,817756,509563,339279,0,0,240371
3083,2023-12-08 19:14:52.372268 UTC,0,96470600,11242419,16267333,38158950,22063286,4381079,1426024,16196283,817756,509563,0,0,0,0
5951,2023-12-08 20:31:06.065188 UTC,2,72046749,8567922,7032987,22870254,21893912,3558389,1111443,10737566,542835,365025,148727,0,0,77699
5952,2023-12-08 20:32:05.961651 UTC,0,96470600,11242419,16267333,38158950,22063286,4381079,1426024,16196283,817756,509563,0,0,0,0


# Multiple explanations for differences between IMEM and instruction validator detections
## Significant less detections by IV compared to IMEM

## Significant more detections by IV compared to IMEM